In [ ]:
import torch

import torch.nn as nn

from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim

from train_functions import get_data, train_model, check

from v2_ResNetE_Decoder import ResNetE_Decoder

In [ ]:
H, W = 128, 128
BATCH_SIZE = 64
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


train_transf = transforms.Compose([transforms.RandomRotation(degrees=5),
    transforms.RandomCrop(480), 
    transforms.ColorJitter(brightness = [0.8,1.2]),
    transforms.Resize((H, W)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()]
)

test_transf = transforms.Compose([transforms.Resize((H, W)),
                             transforms.ToTensor()])

train_dataset, test_dataset = get_data(reduced=True, train_transformation=train_transf, test_transformation=test_transf)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


print("The device is:", DEVICE)
print("Number of train images:", len(train_dataset))
print("Number of test images:", len(test_dataset))
print("Image shape:", train_dataset[0][0].shape)
print("Mask shape:", train_dataset[0][1].shape)

In [ ]:
model = ResNetE_Decoder()

optimizer = optim.Adam(model.parameters())

criterion = nn.L1Loss()
scores = [criterion, nn.MSELoss()]

num_epochs = 50

In [ ]:
path_to_weights = './checkpoints/ResNetE_Decoder/v2_trained_on_27.pth'
state_dict = torch.load(path_to_weights, map_location=torch.device(DEVICE))
model.load_state_dict(state_dict['model_state_dict'])

In [8]:
trained_model, predictions, train_loss, test_loss = train_model(model, train_loader, test_loader, 
                                                                num_epochs, DEVICE, criterion, scores, optimizer)

Batch [10/18], Loss: 0.0496
Batch [11/18], Loss: 0.0489
Batch [12/18], Loss: 0.0499
Batch [13/18], Loss: 0.0662
Batch [14/18], Loss: 0.0491
Batch [15/18], Loss: 0.0466
Batch [16/18], Loss: 0.0492
Batch [17/18], Loss: 0.0490
Batch [18/18], Loss: 0.0535
Epoch [38/50], Train Loss: 0.0521
Test Loss: L1Loss(): 0.1108; MSELoss(): 0.0223
Batch [1/18], Loss: 0.0491
Batch [2/18], Loss: 0.0437
Batch [3/18], Loss: 0.0449
Batch [4/18], Loss: 0.0434
Batch [5/18], Loss: 0.0484
Batch [6/18], Loss: 0.0497
Batch [7/18], Loss: 0.0514
Batch [8/18], Loss: 0.0465
Batch [9/18], Loss: 0.0510
Batch [10/18], Loss: 0.0418
Batch [11/18], Loss: 0.0520
Batch [12/18], Loss: 0.0452
Batch [13/18], Loss: 0.0646
Batch [14/18], Loss: 0.0466
Batch [15/18], Loss: 0.0448
Batch [16/18], Loss: 0.0469
Batch [17/18], Loss: 0.0502
Batch [18/18], Loss: 0.0490
Epoch [39/50], Train Loss: 0.0483
Test Loss: L1Loss(): 0.1109; MSELoss(): 0.0218
Batch [1/18], Loss: 0.0441
Batch [2/18], Loss: 0.0498
Batch [3/18], Loss: 0.0505
Batch [4/1

KeyboardInterrupt: 

In [ ]:
for i in range(10):
    image, mask = test_dataset[i][0], test_dataset[i][1]
    prediction = predictions[i]
    check(image, mask, prediction)